# Accuracy: Pitfalls and Edge Cases

This notebook describes OpenDP's accuracy calculations, and ways in which an analyst might be tripped up by them.

### Overview

#### Accuracy vs. Confidence Intervals

Each privatizing mechanism (e.g. Laplace, Gaussian) in OpenDP has an associated accuracy that is a function of alpha and
either sigma or accuracy. Imagine you have data $D$, and you want, for some function $\phi$ to return $\phi(D)$ in a
differentially private way -- we will call this value $\phi_{dp}(D)$. An $\alpha$-level accuracy guarantee $a$ promises
that, over infinite runs of the privatizing mechanism on the data in question,
$$ \phi(D) \in [\phi_{dp}(D) - a, \phi_{dp}(D) + a] $$
with probability $1 - \alpha$.

This looks very much like the traditional confidence interval, but it is important to note a major difference. In a
canonical confidence interval, the uncertainty being represented is due to sampling error -- that is, how often will it
be the case that $\phi(P)$ (the value of $\phi$ on the underlying population) is within some range of the realized
$\phi(D)$.

In OpenDP (and differentially private data analysis generally), there is an extra layer of uncertainty due to the noise
added to $\phi(D)$ to produce $\phi_{dp}(D)$. OpenDP's accuracy metrics deal only with the uncertainty of $\phi_{dp}(D)$
 relative to $\phi(D)$ and not the uncertainty of $\phi(D)$ relative to $\phi(P)$.

#### What is $D$?

OpenDP allows for analysis of data with an unknown number of rows by resizing the data to ensure consistency with an
estimated size
(see the [unknown dataset size notebook](https://github.com/opendp/opendp/blob/main/python/example/unknown_dataset_size.ipynb)
for more details). Accuracy guarantees are always relative to the resized data $\tilde{D}$, not the data of unknown size.

#### Synopsis

Let's say an analyst releases $\phi_{dp}(D)$ and gets an accuracy guarantee of $a$ at accuracy-level $\alpha$. $D$
is a dataset of unknown size drawn from population $P$ and will be resized to $\tilde{D}$. This suggests that over
infinite runs of this procedure,

- $\phi_{dp}(D) \in [\phi(\tilde{D}) - a, \phi(\tilde{D}) + a]$ with probability $1 - \alpha$
- It is likely that $\phi_{dp}(D) \in [\phi(D) - a, \phi(D) + a]$ with probability $\approx 1 - \alpha$, though we
cannot make any guarantee. For many cases (e.g. resizing the data based on $n$ obtained from a differentially private
count and reasonable bounds on the data elements), this is likely to be approximately true. In the next section, we will
explore some examples of cases where this statement holds to varying extents.

- We cannot directly make statements about the relationship uncertainty of $\phi_{dp}(D)$ relative to $\phi(P)$.

### Accuracy Guarantees In Practice

We now move to some empirical evaluations of how well our accuracy guarantees translate from $\phi(\tilde{D})$ to
$\phi(D)$. We first consider the case where we actually know the size of the underlying data and are able to set
plausible lower/upper bounds on `age`.


In [32]:
# load libraries
import os

import numpy as np
import pandas as pd

from opendp.accuracy import laplacian_scale_to_accuracy
from opendp.meas import make_base_laplace
from opendp.mod import enable_features, binary_search, binary_search_param
from opendp.trans import make_split_dataframe, make_cast_default, make_clamp, make_bounded_resize, \
    make_sized_bounded_mean, make_select_column, make_impute_uniform_float, make_cast_inherent

enable_features("contrib")
enable_features("floating-point")

data_path = os.path.join('.', 'data', 'PUMS_california_demographics_1000', 'data.csv')
var_names = ["age", "sex", "educ", "race", "income", "married", "pid"]
D = pd.read_csv(data_path, names=var_names)
age = D.age
D_mean_age = np.mean(age)

FileNotFoundError: [Errno 2] No such file or directory: './data/PUMS_california_demographics_1000/data.csv'

In [ ]:
# establish extra information for this simulation
age_bounds = (0., 100.)
data_size = 1_000
n_sims = 1_000
epsilon = 1.
alpha = 0.05
releases = []

D_tilde = np.clip(D.age, age_bounds[0], age_bounds[1])
D_tilde_mean_age = np.mean(D_tilde)

preprocessor = (
    # Convert data into a dataframe of string columns
    make_split_dataframe(separator=",", col_names=var_names) >>

    # Selects a column of df, Vec<str>
    make_select_column(key="age", TOA=str) >>

    # Cast the column as Vec<float>, and fill nulls with the default value, 0.
    make_cast_default(TIA=str, TOA=float) >>

    # Clamp age values
    make_clamp(bounds=age_bounds) >>

    # Resize the dataset to length `count_release`.
    #     If there are fewer than `count_release` rows in the data, fill with a constant of 20.
    #     If there are more than `count_release` rows in the data, only keep `count_release` rows
    make_bounded_resize(size=data_size, bounds=age_bounds, constant=20.) >>

    # Compute the mean
    make_sized_bounded_mean(size=data_size, bounds=age_bounds)
)
scale = binary_search_param(lambda s: preprocessor >> make_base_laplace(s), 1, epsilon)

measurement = preprocessor >> make_base_laplace(scale)

with open(data_path, 'r') as infile:
    data = infile.read()
    for index in range(n_sims):
        # get DP mean of age
        releases.append(
            measurement(data)
        )
accuracy = laplacian_scale_to_accuracy(scale, alpha)
print('Accuracy interval (with accuracy value {0}) contains the true mean on D_tilde with probability {1}'.format(
    round(accuracy, 4),
    np.mean([(D_tilde_mean_age >= val - accuracy) & (D_tilde_mean_age <= val + accuracy) for val in releases])))

print('Accuracy interval (with accuracy value {0}) contains the true mean on D with probability {1}'.format(
    round(accuracy, 4),
    np.mean([(D_mean_age >= val - accuracy) & (D_mean_age <= val + accuracy) for val in releases])))

|This performance is as expected. $D$ and $\tilde{D}$ are actually the exact same data (the maximum age in the raw data
 is 93, so our clamp to $[0, 100]$ does not change any values, and we know the correct $n$), so our theoretical
 guarantees on $\tilde{D}$ map exactly to gaurantees on $D$.

We now move to a scenario that is still realistic, but where the performance does not translate quite as well. In this
 case, we imagine that the analyst believes the data to be of size 1050 and uses the default imputation within resize
 so that the extra 50 elements are drawn uniformly from $\{ 0, 1, ..., 100 \}$.

Note that our diagnostic testing of $\tilde{D}$ in the code above is not trivial in this case. In the first example, we
knew that clamp/resize did not change the underlying data, so we could predict exactly the data on which the DP mean
 would actually be calculated. This will not be true for the following examples, so we will simulate finding the true
 underlying mean by releasing an extra DP mean with very high epsilon.

In [ ]:
# establish extra information for this simulation
data_size = 1050

n_sims = 1_000
true_D_tilde_mean_ages = []
dp_D_tilde_mean_ages = []

D_tilde = np.clip(D.age, age_bounds[0], age_bounds[1])
D_tilde_mean_age = np.mean(D_tilde)

preprocessor = (
    # Convert data into a dataframe of string columns
    make_split_dataframe(separator=",", col_names=var_names) >>

    # Selects a column of df, Vec<str>
    make_select_column(key="age", TOA=str) >>

    # Cast the column as Vec<float>
    make_cast_inherent(TIA=str, TOA=float) >>

    # TODO: we need to impute missing values sampled from uniform, not a constant (see line 168)
    make_impute_uniform_float(bounds=age_bounds) >>

    # Clamp age values
    make_clamp(bounds=age_bounds) >>

    # Resize the dataset to length `data_size`.
    #     If there are fewer than `data_size` rows in the data, fill with a constant of 20.
    #     If there are more than `data_size` rows in the data, only keep `count_release` rows
    make_bounded_resize(size=data_size, bounds=age_bounds, constant=20.) >>

    # Compute the mean
    make_sized_bounded_mean(size=data_size, bounds=age_bounds)
)

scale = binary_search_param(lambda s: preprocessor >> make_base_laplace(s), 1, epsilon)

measurement = preprocessor >> make_base_laplace(scale)

with open(data_path, 'r') as infile:
    data = infile.read()
    for index in range(n_sims):
        # get DP mean of age
        releases.append(
            measurement(data)
        )
accuracy = laplacian_scale_to_accuracy(scale, alpha)
print('Accuracy interval (with accuracy value {0}) contains the true mean on D_tilde with probability {1}'.format(
    round(accuracy, 4),
    np.mean([(D_tilde_mean_age >= val - accuracy) & (D_tilde_mean_age <= val + accuracy) for val in releases])))

print('Accuracy interval (with accuracy value {0}) contains the true mean on D with probability {1}'.format(
    round(accuracy, 4),
    np.mean([(D_mean_age >= val - accuracy) & (D_mean_age <= val + accuracy) for val in releases])))

The accuracy guarantee still holds on $\tilde{D}$ (as it should), but we now see much worse performance relative to the
true underlying data $D$.